In [23]:
import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import torch
import torch.nn as nn
import plotly.graph_objects as go

In [2]:
pt = PyntCloud.from_file('../data/birmingham_block_0/segmented-cloud-subsampled.ply')

In [3]:
help(pt)

Help on PyntCloud in module pyntcloud.core_class object:

class PyntCloud(builtins.object)
 |  PyntCloud(points, mesh=None, structures=None, **kwargs)
 |  
 |  A Pythonic Point Cloud.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, points, mesh=None, structures=None, **kwargs)
 |      Create PyntCloud.
 |      
 |      Parameters
 |      ----------
 |      points: pd.DataFrame
 |          DataFrame of N rows by M columns.
 |          Each row represents one point of the point cloud.
 |          Each column represents one scalar field associated to its corresponding point.
 |      
 |      mesh: pd.DataFrame or None, optional
 |          Default: None
 |          Triangular mesh associated with points.
 |      
 |      structures: dict, optional
 |          Map key(base.Structure.id) to val(base.Structure)
 |      
 |      kwargs: custom attributes
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  add_scalar_field(self, name, **kwargs)
 |      Add one or multiple column

In [4]:
pt.points

,x,y,z,red,green,blue,scalar_R,scalar_G,scalar_B,scalar_Composite,scalar_Original_cloud_index,scalar_R_#1,scalar_G_#1,scalar_B_#1,scalar_Composite_#1,scalar_Classification
0,340.81250,365.43750,14.099998,103,109,112,103.0,109.0,112.0,108.000000,0.0,103.0,109.0,112.0,108.000000,0.0
1,340.37500,365.43750,14.160004,97,101,105,97.0,101.0,105.0,101.000000,0.0,97.0,101.0,105.0,101.000000,0.0
2,338.06250,365.21875,14.040001,105,112,121,105.0,112.0,121.0,112.666664,0.0,105.0,112.0,121.0,112.666664,0.0
3,341.28125,365.21875,14.020004,96,100,103,96.0,100.0,103.0,99.666664,0.0,96.0,100.0,103.0,99.666664,0.0
4,341.75000,364.96875,14.000000,84,89,91,84.0,89.0,91.0,88.000000,0.0,84.0,89.0,91.0,88.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107847,368.84375,228.31250,17.080002,51,52,51,51.0,52.0,51.0,51.333332,5.0,NaN,NaN,NaN,NaN,1.0
107848,369.18750,227.34375,17.980003,53,64,60,53.0,64.0,60.0,59.000000,5.0,NaN,NaN,NaN,NaN,1.0
107849,369.43750,226.21875,18.870003,91,120,52,91.0,120.0,52.0,87.666664,5.0,NaN,NaN,NaN,NaN,1.0
107850,370.46875,226.90625,14.470001,73,80,53,73.0,80.0,53.0,68.666664,5.0,NaN,NaN,NaN,NaN,1.0


In [5]:
pt.points['label'] = np.astype(pt.points['scalar_Classification'].values, int)

In [6]:
np.unique(pt.points['label'])

array([0, 1, 2, 3, 4, 5])

In [7]:
classes_count = pd.DataFrame(pt.points['label']).value_counts().reset_index().set_index('label')

In [8]:
classes_df = pd.DataFrame.from_dict({'label': [1, 2, 3, 4, 5, 0], 'name': ['building', 'green area', 'car', 'ground', 'tree', 'road']}).set_index('label')
classes_df

,name
label,
1,building
2,green area
3,car
4,ground
5,tree
0,road


In [9]:
counts = classes_df.join(classes_count)

In [10]:
counts

,name,count
label,,
1,building,48377
2,green area,9617
3,car,1883
4,ground,24442
5,tree,8420
0,road,15113


In [11]:
pt.points[['x', 'y', 'z']][:100]

,x,y,z
0,340.81250,365.43750,14.099998
1,340.37500,365.43750,14.160004
2,338.06250,365.21875,14.040001
3,341.28125,365.21875,14.020004
4,341.75000,364.96875,14.000000
...,...,...,...
95,344.06250,358.21875,13.950005
96,342.56250,355.53125,14.080002
97,399.03125,263.62500,15.040001
98,399.21875,263.68750,14.290001


In [17]:
torch.eye(3).repeat(2, 1, 1)

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]])

In [19]:
# pool of size=3, stride=2
m = nn.MaxPool1d(3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)

In [21]:
output.shape

torch.Size([20, 16, 24])

In [22]:
import numpy as np
import torch.nn.functional as F

class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs, n, 3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool) # (bs, 1024)
      xb = F.relu(self.bn4(self.fc1(flat))) # (bs, 512)
      xb = F.relu(self.bn5(self.fc2(xb))) # (bs, 256)
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix


class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2) # why transpose?

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64


class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

Steps

1. Dataset for reading a point cloud
2. ? Applying transformations like rotation? 
3. Subsampling point cloud 

In [ ]:
import numpy as np
import pyvista as pv

# Generate random point cloud data
points = np.random.rand(100, 3)

# Create a PolyData object from the points
point_cloud = pv.PolyData(points)

# Plot the point cloud
plotter = pv.Plotter()
plotter.add_mesh(point_cloud, color="green", point_size=5, render_points_as_spheres=True)
plotter.show()

In [ ]:
class PointSampler:
    
    def __init__(self, subsample_size):
        self.subsample_size = subsample_size
    
    def __call__(self, point_cloud):
        indices = np.random.choice(len(point_cloud), size=self.subsample_size, replace=False)
        sampled_points = point_cloud[indices]
        return sampled_points
    

# Normalize globally or per batch?
class Normalize:
    
    def __call__(self, point_cloud):
        pass 
    
